This notebook basicaly read the **stage** folder in the drive and rename the data that is stored in the folders.

The new name is just a number, but for each image there are a csv explaning each image.


**NOTE:** This is working only for jpg files but is easy to change it

In [1]:
#Just to open the folder in the drive
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive/MyDrive/MAC5768-visao_e_processamento_de_imagens/

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive/MyDrive/MAC5768-visao_e_processamento_de_imagens


In [2]:
import os
import glob
import shutil
import _thread
import threading
from skimage import io
import pandas as pd

In [3]:
class Process(threading.Thread):
  

  def __init__(self, file_path, class_path, count, bg_description):
    threading.Thread.__init__(self)
    self.path = file_path
    self.count=count
    self.class_path = class_path
    self.bg_description = bg_description

  @property
  def csv_full_description(self):
    return self._csv_full_description
  
  def __get_file_size(self):

    size = os.path.getsize(self.path)
    size = size/(1024*1024)
    round(size,4)
    return str(size)

  
  def __image_description(self) -> str:
    
    image = io.imread(self.path)
    rows, cols, c = image.shape
    return f'{rows},{cols}'

  def run(self):
    self._csv_full_description=''

    splited = self.path.split('/')[1:]

    #attributes
    file_name = str(self.count)+'.jpg'
    class_name = splited[0]
    enviroment = splited[1]
    lighting = splited[2]
    object_number = splited[3].replace('object_','')
    background = splited[4].replace('bg','')

    bg_description = (self.bg_description['description']
                          .loc[(self.bg_description['env']==enviroment) &
                               (self.bg_description['bg']==splited[4])]).item()

    soft_description = f'\n{file_name},{class_name},{enviroment},{lighting},{object_number},{background},{bg_description}'

    shutil.copy2(self.path, os.path.join(self.class_path, file_name))

    shape = self.__image_description()
    file_size = self.__get_file_size()
    self._csv_full_description = soft_description+','+shape+','+file_size


In [5]:
stage_path = 'stage'
dataset_path = 'dataset'
classes = os.listdir(stage_path)

csv_full_description = 'file_name,class,enviroment,lighting,object_number,background,bg_description,rows,cols,file_size(MB)'

itens_by_class={}
count = 0 # name for the image

threads = []
for c in classes:
  background_description = pd.read_csv(os.path.join(stage_path,c,'describe.csv'))

  count_by_class = 0
  class_path = os.path.join(dataset_path, 'data',c)
  if not os.path.exists(class_path):
    os.makedirs(class_path)

  # all files in that class
  files = glob.glob(f'{stage_path}/{c}/**/*.jpg', recursive=True)

  for path in files:

    count+=1
    count_by_class+=1
    t=Process(path, class_path, count, background_description)
    t.start()
    threads.append(t)

  
  itens_by_class[c] = count_by_class

for tr in threads:
  tr.join()
  csv_full_description+=tr.csv_full_description

with open(f'{dataset_path}/metadata.csv', 'w') as f:
  f.write(csv_full_description)

print(f'Done! A total of {count} images was processed')
print('\nProcessed by class:\n')
print(*itens_by_class.items(), sep='\n')

Done! A total of 15 images was processed

Processed by class:

('pen', 5)
('medicine', 5)
('toothbrush', 5)


In [ ]:
# stage_path = 'stage'
# dataset_path = 'dataset'
# classes = os.listdir(stage_path)

# csv_full_description = 'file_name,class,enviroment,lighting,object_number,background'

# itens_by_class={}
# total_itens = 0
# for c in classes:
#   count = 0 # name for the image
#   count_itens=0

#   class_path = os.path.join(dataset_path, c)
#   if not os.path.exists(class_path):
#     os.makedirs(class_path)

#   # all files in that class
#   files = glob.glob(f'{stage_path}/{c}/**/*.jpg', recursive=True)

#   for path in files:
#     splited = path.split('/')[1:]
#     count_itens+=1
#     count +=1

#     #attributes
#     file_name = str(count)+'.jpg'
#     class_name = splited[0]
#     enviroment = splited[1]
#     lighting = splited[2]
#     object_number = splited[3].replace('object_','')
#     background = splited[4].replace('bg','')

#     shutil.copy2(path, os.path.join(class_path, file_name))
#     csv_full_description += f'\n{file_name},{class_name},{enviroment},{lighting},{object_number},{background}'
    
#   itens_by_class[c] = count_itens
#   total_itens+=count_itens
# with open(f'{dataset_path}/metadata.csv', 'w') as f:
#   f.write(csv_full_description)

# print(f'Done! A total of {total_itens} images was processed')
# print('\nProcessed by class:\n')
# print(*itens_by_class.items(), sep='\n')